In [1]:
training_data = [
    ["Green", 3, "Mango"],
    ["Yellow", 3, "Mango"],
    ["Red", 1, "Grape"],
    ["Red", 1, "Grape"],
    ["Yellow", 3, "Lemon"],
]
print(training_data)

[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


In [2]:
# Column Labels.
# These are used to pront the tree.
header = ["color", "diameter", "label"]
print(header)

['color', 'diameter', 'label']


In [3]:
def unique_vals(rows, col):
    """find the unique values for a coulmn in dataset."""
    return set([row[col] for row in rows])

In [4]:
print(unique_vals(training_data, 0))

{'Yellow', 'Green', 'Red'}


In [5]:
print(unique_vals(training_data, 1))

{1, 3}


In [6]:
def class_count(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}
    for row in rows:
        # In our dataset format, the labesl is alwasy the last column.
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
class_count(training_data)

{'Mango': 2, 'Grape': 2, 'Lemon': 1}

In [8]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [9]:
is_numeric(7)

True

In [10]:
is_numeric("Red")

False

In [11]:
class Question:
    """
    A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for color) and a 
    'column value' (e.g., Green). The 'match' method is used to compare 
    the feature value in an example to the feature value stored in the question.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # Compare the feature value in example to feature value in this question
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        # This is just a method to print the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (header[self.column], condition, str(self.value))
    

In [12]:
def partition(rows, question):
    """
    Partitions a dataset.

    For each row in the dataset, check if it matches the question.
    If so, add it to 'true rows', otherwise add it to 'false rows'
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

######
# Demo:
# Lets partition the training data based on whether rows are Red
######
    
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Mango'], ['Yellow', 3, 'Mango'], ['Yellow', 3, 'Lemon']]


In [13]:
def gini(rows):
    """
    Calculate the Gini Impurity for a list of rows.
    
    There are few different ways to do it. This is the most concise. 
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_count(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

#####
# Demo:
#
# First, we'll look at a dataset with no mixing.
no_mixing = [["Mango"], ["Mango"]]
print(gini(no_mixing))

# Now, we'll look at a dataset with lot of mixing.
lot_of_mixing = [["Mango"], ["Apple"], ["Kiwi"], ["Blueberry"], ["Cranberry"]]
print(gini(lot_of_mixing))

0.0
0.7999999999999998


In [14]:
def info_gain(left, right, current_uncertainty):
    """
    Information Gain - The uncertainty of the starting node, minus the weighted impurity of two child nodes.
    """
    
    p = float(len(left) / len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

#####
# Demo:

# Calculate the uncertainty of our trained data.
current_uncertainty = gini(training_data)
print(current_uncertainty)

# How much information we gain by partitioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
information_gain = info_gain(true_rows, false_rows, current_uncertainty)
print(information_gain)

# How much information we gain by partitioning on 'Red'?
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
information_gain = info_gain(true_rows, false_rows, current_uncertainty)
print(information_gain)



0.6399999999999999
3.1399999999999997
1.9733333333333334


In [15]:
def find_best_split(rows):
    """
    Find the best question to ask by iterating over every feature/value and calculating the information gain.
    """
    best_gain = 0 # Keep track of the best information gain
    best_question = None # Keep track of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # Number of columns
    
    for col in range(n_features): # For each feature
        
        values = set([row[col] for row in rows]) # Unique values in the column
        
        for val in values: # For each value
            
            question = Question(col, val)
            
            # Try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # Skip this split of it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            # We can actually use '>' instead of '>=' here
            if gain >= best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question
    
best_gain, best_question = find_best_split(training_data)
print(best_gain)
print(best_question)

3.1399999999999997
Is color == Green?


In [16]:
class Leaf:
    """
    A Leaf node classifies data.
    This holds a dictionary of class (e.g., 'Apple') -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    
    def __init__(self, rows):
        self.predictions = class_count(rows)

In [17]:
class Decision_Node:
    """
    A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch        

In [18]:
def build_tree(rows):
    """
    Builds the tree.
    
    Rules of recursion 
    1) Believe that it works. 
    2) Start by checking fpr the base case (no further information gain).    
    """
    
    # Try partitioning the dataset on each of the unique attribute,
    # Calculate the information gain,
    # And return the question that produces the higehest information gain
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain.
    # Since we can ask no further questions we'll return a leaf
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch
    true_branch = build_tree(true_rows)

    # Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point, as well as the branches to follow
    # depending on the answer
    
    return Decision_Node(question, true_branch, false_branch)

In [19]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function"""
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # Print the question at this point
    print(spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print(spacing + "--> True:")
    print_tree(node.true_branch, spacing + "  ")
    
    # Call this function recursively on the false branch
    print(spacing + "--> False:")
    print_tree(node.false_branch, spacing + "  ")    

In [20]:
def classify(row, node):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node to the example we're considering
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

my_tree = build_tree(training_data)
my_classification = classify(training_data[0], my_tree)

print(my_tree)
print(my_classification)

{'Mango': 1}


In [24]:
def print_leaf(counts):
    """Print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

my_leaf_0 = print_leaf(classify(training_data[0], my_tree))
my_leaf_1 = print_leaf(classify(training_data[1], my_tree))

print(my_leaf_0)
print(my_leaf_1)

{'Mango': '100%'}
{'Mango': '50%', 'Lemon': '50%'}


In [26]:
if __name__ == "__main__":
    
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    # Evaluate
    training_data = [
    ["Green", 3, "Mango"],
    ["Yellow", 3, "Mango"],
    ["Red", 1, "Grape"],
    ["Red", 1, "Grape"],
    ["Yellow", 3, "Lemon"],
    ]
    
    for row in training_data:
        print("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))
        
    


Is color == Green?
--> True:
  Predict {'Mango': 1}
--> False:
  Is color == Red?
  --> True:
    Predict {'Grape': 2}
  --> False:
    Predict {'Mango': 1, 'Lemon': 1}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}
